In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

import calendar


In [2]:
# Initilaize an instance
app = Flask(__name__, template_folder="build", static_url_path='/build')

# Database Setup
app.config['SECRET_KEY'] = 'Thisissupposedtobesecret!'
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/coned.sqlite"

In [3]:
# app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()

/anaconda3/envs/PythonData/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [4]:
    df = pd.read_csv("final_leaks.csv")

    ## Convert the Date column to correct datetime format
    df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')

    ## get the data for the full year where date = 2013
    year_data = df[df['Date'].dt.year == 2013]

    ## get data for leaks per zipcode for that given year
    year_zip_data = pd.DataFrame(year_data.groupby('ZIP_CODE').sum()['TMAX'])
    year_zip_data.rename(columns = {'TMAX': 'Total_Leaks'}, inplace=True)
    year_data_dict = year_zip_data.to_dict()

    # set variable for monthly data
    monthly_data = year_data[year_data['Date'].dt.month == 1]
    ## get monthly data of temperatures for given month
    ## test to see if it filters out by month. 1 = January
    ## monthly_temp_data = year_data[year_data['Date'].dt.month == 1]
    monthly_temp_data = monthly_data[['Date', 'TMAX']]
    monthly_temp_data.groupby('Date')['TMAX'].max()
    ## put grouped data into its on DF
    grouped_month_temp_data = pd.DataFrame(monthly_temp_data.groupby('Date')['TMAX'].max())
    ## rename columns, then reset eindex
    grouped_month_temp_data.rename(columns = {'TMAX': 'Temp'}, inplace=True)
    grouped_month_temp_data.reset_index(level=0, inplace=True)
    ## needed to change date to string because jsonify does not support datetime values
    grouped_month_temp_data['Date'] = grouped_month_temp_data['Date'].astype(str)
    ## made the keys for the days of the month start at 1 instead of 0
    grouped_month_temp_data.index = np.arange(1,len(grouped_month_temp_data)+1)
    ## converted DF to dictionary 
    month_data_dict = grouped_month_temp_data.to_dict()
    ## Accessed the data
    final_month_dict = month_data_dict['Temp']

    ## get monthly data of leak counts per day for given month
    monthly_leaks = pd.DataFrame(monthly_data.groupby('Date')['TMAX'].count())
    monthly_leaks.rename(columns = {'TMAX': 'Number_of_Leaks'}, inplace=True)
    monthly_leaks.reset_index(level=0, inplace=True)
    monthly_leaks['Date'] = monthly_leaks['Date'].astype(str)
    monthly_leaks.index = np.arange(1,len(monthly_leaks)+1)
    monthly_leaks_dict = monthly_leaks.to_dict()
    final_monthly_leaks_dict = monthly_leaks_dict['Number_of_Leaks']


    # Please the dictionaries from above into final dictionary to respond to API request
    final_dict = {
        "ZipData:" : year_data_dict,
        "TempData" : final_month_dict,
        "MonthlyLeaks" : final_monthly_leaks_dict
    }

In [5]:
grouped_month_temp_data

Date  Temp
1   2013-01-01    40
2   2013-01-02    33
3   2013-01-03    32
4   2013-01-04    37
5   2013-01-05    42
6   2013-01-06    46
7   2013-01-07    45
8   2013-01-08    48
9   2013-01-09    49
10  2013-01-10    47
11  2013-01-11    46
12  2013-01-12    47
13  2013-01-13    50
14  2013-01-14    56
15  2013-01-15    38
16  2013-01-16    37
17  2013-01-17    43
18  2013-01-18    35
19  2013-01-19    51
20  2013-01-20    53
21  2013-01-21    32
22  2013-01-22    27
23  2013-01-23    20
24  2013-01-24    22
25  2013-01-25    24
26  2013-01-26    27
27  2013-01-27    34
28  2013-01-28    36
29  2013-01-29    49
30  2013-01-30    59
31  2013-01-31    61

In [6]:
leaks_dict = {}
for i in range(6):
    wh_year = 2013+i
    year_data = df[df['Date'].dt.year == wh_year]
    
    ## get data for leaks per zipcode for that given year
    year_zip_data = pd.DataFrame(year_data.groupby('ZIP_CODE').sum()['TMAX'])
    year_zip_data.rename(columns = {'TMAX': 'Total_Leaks'}, inplace=True)
#     print(i)
#     print(year_zip_data.head())
    year_zip_data.reset_index(inplace=True)
    json = year_zip_data.to_json(orient='records')
    leaks_dict[wh_year] = json


In [7]:
year_data = df[df['Date'].dt.year == wh_year]
year_zip_data = pd.DataFrame(year_data.groupby('ZIP_CODE').sum()['TMAX'])
year_zip_data.rename(columns = {'TMAX': 'Total_Leaks'}, inplace=True)
#     print(i)
#     print(year_zip_data.head())
leaks_dict[wh_year] = year_zip_data

year_zip_data.reset_index()

ZIP_CODE  Total_Leaks
0       10001         1916
1       10002         9651
2       10003         4693
3       10004          195
4       10005          510
5       10006          348
6       10007         1549
7       10009         9165
8       10010         2014
9       10011         6129
10      10012         4714
11      10013         1992
12      10014         3644
13      10016         2462
14      10017          704
15      10018          958
16      10019         4775
17      10020           78
18      10021         6311
19      10022         1738
20      10023         3441
21      10024         5622
22      10025         7860
23      10026         6257
24      10027         9060
25      10028         2735
26      10029        13343
27      10030         4507
28      10031         6181
29      10032         6299
..        ...          ...
143     11379          519
144     11385         3204
145     11411          950
146     11412         1389
147     11413         1805
148     11414         1596
149     11415          359
150     11416          523
151     11417          685
152     11418         1239
153     11419         1178
154     11420         1934
155     11421         1320
156     11422         1615
157     11423         1165
158     11426         1771
159     11427         1101
160     11428          178
161     11429          994
162     11432         2352
163     11433         3217
164     11434         2947
165     11435          889
166     11436         1085
167     11691         3762
168     11692         1174
169     11693          912
170     11694          735
171     11697          430
172     99999         2991

[173 rows x 2 columns]

In [ ]:
leaks_dict[2014]

In [ ]:
df

In [8]:
months = pd.DataFrame(df['Date'].dt.month)
months.rename(columns = {"Date": 'Month'}, inplace=True)


In [9]:
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

In [ ]:
df

In [10]:
df['Month'] = df['Month'].apply(lambda x: calendar.month_abbr[x])
grouped_month_day_df  = df.groupby(['Year','Month', 'Date'], sort=False).count()['TMAX']

In [11]:
grouped_month_day_df = pd.DataFrame(grouped_month_day_df)
grouped_month_day_df.head()

TMAX
Year Month Date            
2013 Jan   2013-01-01    35
           2013-01-02    46
           2013-01-03    44
           2013-01-04    33
           2013-01-05    53

In [13]:
grouped_month_day_indexed_df = grouped_month_day_df.reset_index()
grouped_month_day_indexed_df['Date'] = grouped_month_day_indexed_df['Date'].astype(str)
grouped_month_day_indexed_df.head()

Year Month        Date  TMAX
0  2013   Jan  2013-01-01    35
1  2013   Jan  2013-01-02    46
2  2013   Jan  2013-01-03    44
3  2013   Jan  2013-01-04    33
4  2013   Jan  2013-01-05    53

In [15]:
response = {}
for row in grouped_month_day_indexed_df.values:
    
    year = row[0]
    month = row[1]
    date = row[2]
    tmax = row[3]
    
    
    if year not in response:
        response[year] = {}
    else:
        if month not in response[year]:
            response[year][month] = {}
        else:
            if date not in response[year][month]:
                response[year][month][date] = tmax
            else:
                pass

response

{2013: {'Jan': {'2013-01-03': 44,
   '2013-01-04': 33,
   '2013-01-05': 53,
   '2013-01-06': 42,
   '2013-01-07': 55,
   '2013-01-08': 40,
   '2013-01-09': 35,
   '2013-01-10': 40,
   '2013-01-11': 50,
   '2013-01-12': 34,
   '2013-01-13': 33,
   '2013-01-14': 48,
   '2013-01-15': 43,
   '2013-01-16': 40,
   '2013-01-17': 40,
   '2013-01-18': 30,
   '2013-01-19': 45,
   '2013-01-20': 30,
   '2013-01-21': 40,
   '2013-01-22': 35,
   '2013-01-23': 43,
   '2013-01-24': 60,
   '2013-01-25': 51,
   '2013-01-26': 30,
   '2013-01-27': 40,
   '2013-01-28': 39,
   '2013-01-29': 38,
   '2013-01-30': 48,
   '2013-01-31': 27},
  'Feb': {'2013-02-02': 57,
   '2013-02-03': 45,
   '2013-02-04': 45,
   '2013-02-05': 40,
   '2013-02-06': 43,
   '2013-02-07': 41,
   '2013-02-08': 41,
   '2013-02-09': 46,
   '2013-02-10': 41,
   '2013-02-11': 47,
   '2013-02-12': 44,
   '2013-02-13': 59,
   '2013-02-14': 41,
   '2013-02-15': 42,
   '2013-02-16': 37,
   '2013-02-17': 28,
   '2013-02-18': 52,
   '2013-02-1

In [ ]:
grouped_month_day_df.to_json(orient='records')

In [ ]:
# dic = {}

# for year in df['Year'].unique():
#     dic[year] = []
#     for month in df['Month'].unique():
#         dic[year][month] = month
#         if grouped_month_day_df.loc[df['Year'] == year and df.loc[df['Month'] == month]:
#                   dic[year][month][date] = grouped_month_day_df['TMAX']
                  
        